In [ ]:
import tensorflow as tf
from tensorflow.keras import layers
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
IMAGE_SIZE = 256
BATCH_SIZE = 32
CHANNELS = 3
EPOCHS = 30

In [ ]:
# Load the dataset using image_dataset_from_directory
dataset = tf.keras.preprocessing.image_dataset_from_directory(
    "beansLeaf",
    seed=123,
    shuffle=True,
    image_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE
)

In [ ]:
# Convert dataset to NumPy arrays
images = []
labels = []
for image_batch, label_batch in dataset:
    images.append(image_batch.numpy())
    labels.append(label_batch.numpy())
images = tf.concat(images, axis=0)
labels = tf.concat(labels, axis=0)

In [ ]:
# Create an instance of the ImageDataGenerator with desired augmentation options
data_augmentation = ImageDataGenerator(
    rotation_range=10,
    zoom_range=0.1,
    horizontal_flip=True,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.2,
    vertical_flip=False,
    fill_mode='nearest'
)

In [ ]:
# Create a generator function for augmented image batches and labels
def augmented_data_generator():
    for image_batch, label_batch in data_augmentation.flow(images, labels, batch_size=BATCH_SIZE, shuffle=False):
        yield image_batch, label_batch

In [ ]:
# Create a new dataset from the generator function
augmented_dataset = tf.data.Dataset.from_generator(
    augmented_data_generator,
    output_signature=(
        tf.TensorSpec(shape=(BATCH_SIZE, IMAGE_SIZE, IMAGE_SIZE, CHANNELS), dtype=tf.float32),
        tf.TensorSpec(shape=(BATCH_SIZE,), dtype=tf.int32)
    )
)


In [ ]:
# Concatenate the augmented dataset and original dataset
combined_dataset = tf.data.Dataset.concatenate(augmented_dataset, dataset)


In [ ]:
# Shuffle and batch the combined dataset
combined_dataset = combined_dataset.shuffle(len(images)).batch(BATCH_SIZE)

In [ ]:
# Split the combined dataset into training and validation sets
val_size = int(0.2 * len(images))
train_dataset = combined_dataset.skip(val_size)
val_dataset = combined_dataset.take(val_size)

In [ ]:
# Normalize the pixel values to the range [0, 1]
normalization_layer = layers.experimental.preprocessing.Rescaling(1./255)

In [ ]:
# Apply normalization to the datasets
train_dataset = train_dataset.map(lambda x, y: (normalization_layer(x), y))
val_dataset = val_dataset.map(lambda x, y: (normalization_layer(x), y))

In [ ]:
# Define the Transformer model
class TransformerModel(tf.keras.Model):
    def __init__(self, num_classes):
        super(TransformerModel, self).__init__()
        self.transformer = layers.Transformer(
            num_layers=4,
            d_model=128,
            num_heads=8,
            d_ff=256,
            input_shape=(IMAGE_SIZE, IMAGE_SIZE, CHANNELS),
            output_sequence_length=1,
            dropout=0.1
        )
        self.flatten = layers.Flatten()
        self.dense = layers.Dense(num_classes, activation='softmax')
        
    def call(self, inputs):
        x = self.transformer(inputs)
        x = self.flatten(x)
        x = self.dense(x)
        return x

num_classes = len(dataset.class_names)
model = TransformerModel(num_classes)


In [ ]:
model.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy']
)


In [ ]:
history = model.fit(train_dataset, validation_data=val_dataset, epochs=EPOCHS)


In [ ]:
test_loss, test_accuracy = model.evaluate(val_dataset)
print(f"Test Loss: {test_loss:.2f}")
print(f"Test Accuracy: {test_accuracy:.2f}")

In [ ]:
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()